In [13]:
import pandas as pd
from pycaret.classification import *

In [1]:
# Retrieve Training and Testing Data 
df_train = pd.read_csv('data/trainClean.csv')
df_test = pd.read_csv('data/testClean.csv')

#Define Catergorical and Numerical features
cat_features = ['HomePlanet', 'CryoSleep', 'Deck', 'Num', 'Side', 'Destination', 'Age', 'VIP', 'FirstName', 'LastName',]
num_features = ['PassengerId', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

#PassengerId	HomePlanet	CryoSleep	Deck	Num	Side	Destination	Age	VIP	RoomService	FoodCourt	ShoppingMall	FirstName	LastName	Spa	VRDeck	Transported

NameError: name 'pd' is not defined

In [2]:
#Setup PyCaret with training dataset, target variable, and categorical and numerical features
test = setup(df_train, target='Transported', categorical_features=cat_features, numeric_features=num_features)

#Compare different models accuracy on given dataset
best_model= compare_models()

#Example Logistic Regression
#lr_model = create_model('lr')
#tuned_lr = tune_model(lr_model, n_iter=10, search_library='optuna', search_algorithm='tpe', choose_better = True)

#Tuner hyperparameters by passing the best model found for the particular dataset, number of iterations, and the search method you want to use
#Increase number of iterations to make the search run longer and increase accuracy
tuned_model = tune_model(best_model, n_iter=10, search_library='optuna', search_algorithm='tpe', choose_better = True)

# plot feature importance
plot_model(best_model, plot = 'feature')

NameError: name 'setup' is not defined

In [16]:
pred = predict_model(tuned_model, data=df_test)
pred

,PassengerId,HomePlanet,CryoSleep,Deck,Num,Side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,FirstName,LastName,Spa,VRDeck,prediction_label,prediction_score
0,0013_01,1,1,7,3,2,3,27.0,0,0.0,0.0,0.0,1351,2359,0.0,0.0,0,0.5724
1,0018_01,1,0,6,4,2,3,19.0,0,0.0,9.0,0.0,1426,678,2823.0,0.0,0,0.9792
2,0019_01,3,1,3,0,2,1,31.0,0,0.0,0.0,0.0,13,609,0.0,0.0,1,0.9642
3,0021_01,3,0,3,1,2,3,38.0,0,0.0,6652.0,0.0,1454,1456,181.0,585.0,1,0.9555
4,0023_01,1,0,6,5,2,3,20.0,0,10.0,0.0,635.0,745,1537,0.0,0.0,1,0.5663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,1,7,1496,2,3,34.0,0,0.0,0.0,0.0,988,646,0.0,0.0,1,0.5982
4273,9269_01,1,0,0,0,0,3,42.0,0,0.0,847.0,17.0,1402,1299,10.0,144.0,1,0.5428
4274,9271_01,2,1,4,296,1,1,-1.0,0,0.0,0.0,0.0,625,2267,0.0,0.0,1,0.9814
4275,9273_01,3,0,4,297,1,0,-1.0,0,0.0,2680.0,0.0,1317,1393,0.0,523.0,1,0.8550


In [17]:
# Map Predictions to Binary Values
tfMap = { 0: False, 1: True }
pred['Transported'] = [tfMap[i] for i in pred['prediction_label']]

# Add PassengerId Column to Predictions
kaggleCSV = pd.merge(df_test[['PassengerId']], pred[['Transported']], left_index=True, right_index=True)

# Save Predictions to CSV
kaggleCSV.to_csv('results/example_results.csv', index=False)